## Example: Computing Risk Neutral Probabilities in the Binomial Lattice
The expected value of the future risk neutral price in a one-period binomial model
is:

$$
\begin{equation*}
    \mathcal{D}_{1,0}(\bar{r})\cdot{S_{\circ}} = \mathbb{E}_{\mathbb{Q}}\left(S_{1}\right)
\end{equation*}
$$

where $\mathcal{D}_{1,0}(\bar{r})$ is the continuous discount factor between period $0\rightarrow{1}$, 
and $\bar{r}$ is the annualized risk-free rate. The expectation operator $\mathbb{E}_{\mathbb{Q}}(\dots)$ is taken 
with respect to a risk neutral probability measure $\mathbb{Q}$.
Thus, the expectation operator $\mathbb{E}_{\mathbb{Q}}(\dots)$ is:

$$
\begin{equation*}
\mathcal{D}_{1,0}(\bar{r})\cdot{S_{\circ}} = q\cdot{S^{u}} + (1-q)\cdot{S^{d}}
\end{equation*}
$$

where $q$ is the risk neutral probability of the $\texttt{up}$ state, 
and the share price in the $\texttt{up}$ and $\texttt{down}$ states are the product of an $\texttt{up}$ factor $u$ (or a $\texttt{down}$ factor $d$) and the initial share price, i.e., $S^{u} = u\cdot{S_{\circ}}$ and $S^{d} = d\cdot{S_{\circ}}$. Putting everything together, gives:

$$
\begin{equation*}
q = \frac{\mathcal{D}_{1,0}(\bar{r}) - d}{u - d}
\end{equation*}
$$

### Learning objective
In this example, we will familiarize students with the computation of the real-world and risk-neutral probability parameters for a binomial lattice model. In particular, we will:

* Load the historical dataset. We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) for the past five-trading years (a maximum of `1256` data points per firm). 
* Compute the $(p,q,u,d)$ parameters for each firm in the dataset. The results will be stored in `DataFrame`

## Setup
We set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our example problem.

In [1]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5660-Examples-F23`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`


securityterm (generic function with 1 method)

### Constants

In [7]:
number_of_trading_days = 1256;
Δt = (1.0/252);
risk_free_rate = 0.045;
𝒟(r,t) = exp(r*t);

## Load historical data set
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) for the past five-trading years (a maximum of `1256` data points per firm). However, not all the firms in the `dataset` have the maximum number of trading days, i.e., some firms are missing information for various reasons; perhaps they were acquired, merged, or delisted, etc. We will exclude these firms from the `dataset`. We load the price `dataset` by calling the `MyPortfolioDataSet()` function:

In [2]:
dataset = MyPortfolioDataSet() |> x-> x["dataset"];

The `list_of_all_firms` array holds the list of firm indexes in the dataset that have complete data, i.e., all `1256` data values: 

In [3]:
list_of_all_firms = keys(dataset) |> collect |> sort;

While it is sometimes convenient to work with the data using the `firm_index`, often we specify the [ticker symbol](https://en.wikipedia.org/wiki/Ticker_symbol#:~:text=A%20ticker%20symbol%20or%20stock,on%20a%20particular%20stock%20market.) instead. To facilitate this, let's load a mapping between the `firm_index` and the ticker symbols using the `MyFirmMappingDataSet()` function. We store this mapping in the `firm_mapping_df` variable, which is of type `DataFrame`:

In [4]:
firm_mapping_df = MyFirmMappingDataSet()

505×3 DataFrame
 Row │ Symbol   Name                    Sector                 
     │ String7  String                  String31               
─────┼─────────────────────────────────────────────────────────
   1 │ MMM      3M                      Industrials
   2 │ AOS      A. O. Smith             Industrials
   3 │ ABT      Abbott Laboratories     Health Care
   4 │ ABBV     AbbVie                  Health Care
   5 │ ABMD     Abiomed                 Health Care
   6 │ ACN      Accenture               Information Technology
   7 │ ATVI     Activision Blizzard     Communication Services
   8 │ ADM      ADM                     Consumer Staples
   9 │ ADBE     Adobe                   Information Technology
  10 │ AAP      Advance Auto Parts      Consumer Discretionary
  11 │ AMD      Advanced Micro Devices  Information Technology
  ⋮  │    ⋮               ⋮                       ⋮
 496 │ WLTW     Willis Towers Watson    Financials
 497 │ WYNN     Wynn Resorts            Consumer Discretionary
 498 │ XEL      Xcel Energy             Utilities
 499 │ XLNX     Xilinx                  Information Technology
 500 │ XYL      Xylem                   Industrials
 501 │ YUM      Yum! Brands             Consumer Discretionary
 502 │ ZBRA     Zebra Technologies      Information Technology
 503 │ ZBH      Zimmer Biomet           Health Care
 504 │ ZION     Zions Bancorp           Financials
 505 │ ZTS      Zoetis                  Health Care
                                               484 rows omitted

## Estimate the up, down, p, and q values for each firm in the dataset
After validating our lattice implementation, it's time to utilize historical data to compute a share price prediction. To create a binomial lattice model for future share prices, we need to estimate three critical parameters: $p$, $u$, and $d$.

* The $p$ parameter represents the __real world__ probability of a share price increase or an `up` move between two periods $j\rightarrow{j+1}$. As a binary lattice model only allows `up` and `down` moves, the probability of a `down` move is $1-p$.
* The $q$ parameter represents the __risk neutral__ probability of a share price increase or an `up` move between two periods $j\rightarrow{j+1}$. As a binary lattice model only allows `up` and `down` moves, the probability of a `down` move is $1-q$.
* The $u$ parameter represents the amount of an `up` move. If $S_{j}$ stands for the share price in period $j$, and $S_{j+1}$ is the share price in the next period, then an `up` move will give $S_{j+1} = u\cdot{S}_{j}$.
* The $d$ parameter represents the amount of a `down` move. If $S_{j}$ stands for the share price in period $j$, and $S_{j+1}$ is the share price in the next period, then a `down` move will give $S_{j+1} = d\cdot{S}_{j}$.

### Implementation
To compute the $(p,q,u,d)$ values for each firm in the dataset, first, we initialize the `binomial_model_parameter_table` as an empty `DataFrame`, the `binomial_model_parameter_table` will hold the tuple of values in table format. Next, we iterate through each firm in the `list_of_all_firms`, the sorted list of keys from the `dataset` dictionary, using a `for` loop. For each iteration of the loop, we:

* Get the `firm_index`, `firm_ticker` and the `firm_data` for a particular firm
* Then we populate the `log_growth_array` with the daily growth rate values (growth rate is the instantaneous return dived by the time-step)
* Next, we call the `analyze(...)` function with the `log_growth_array` and timestep `Δt` as parameters. The `analyze(...)` function returns the estimated magnitude of the `u` and `d` factors as well as the probability `p` of an `up` move
* We compute the risk-neutral probability `q`
* We store the values in the `firm_results_tuple` and add this data to the `binomial_model_parameter_table` using the `push!(...)` function. 

In [13]:
binomial_model_parameter_table = DataFrame();
for i ∈ eachindex(list_of_all_firms)
    
    # get the firm index, and ticker
    firm_index = list_of_all_firms[i];
    firm_ticker = firm_mapping_df[firm_index, :Symbol];

    # grab the dataset for this firm, and compute the return
    firm_dataset = dataset[firm_index];
    log_growth_array = Array{Float64,1}(undef, number_of_trading_days-1)
    for j ∈ 2:number_of_trading_days
    
        S₁ = firm_dataset[j-1,:volume_weighted_average_price];
        S₂ = firm_dataset[j,:volume_weighted_average_price];
        log_growth_array[j-1] = (1/Δt)*log(S₂/S₁);
    end
    
    # analyze the returns, compute u, d and p -
    (u,d,p) = analyze(log_growth_array, Δt = Δt);
    
    # compute the risk neutral probability -
    q = (𝒟(risk_free_rate, Δt) - d)/(u - d);
    
    # store -
    firm_results_tuple = (
        firm_index=firm_index, firm_ticker=firm_ticker, p = p, q = q, u = u, d = d
    );
    push!(binomial_model_parameter_table, firm_results_tuple)
end

In [14]:
binomial_model_parameter_table

458×6 DataFrame
 Row │ firm_index  firm_ticker  p         q         u        d        
     │ Int64       String7      Float64   Float64   Float64  Float64  
─────┼────────────────────────────────────────────────────────────────
   1 │          1  MMM          0.516335  0.545115  1.00972  0.988739
   2 │          2  AOS          0.506773  0.510835  1.01228  0.987544
   3 │          3  ABT          0.542629  0.522264  1.01006  0.989375
   4 │          4  ABBV         0.525896  0.510608  1.01125  0.988624
   5 │          6  ACN          0.54741   0.529822  1.01025  0.988832
   6 │          7  ATVI         0.503586  0.498259  1.01277  0.98767
   7 │          8  ADM          0.54741   0.518992  1.01042  0.989132
   8 │          9  ADBE         0.563347  0.545444  1.01286  0.984959
   9 │         10  AAP          0.513944  0.504051  1.01321  0.986934
  10 │         11  AMD          0.533068  0.49607   1.02216  0.978541
  11 │         12  AES          0.549004  0.519471  1.01321  0.986094
  ⋮  │     ⋮            ⋮          ⋮         ⋮         ⋮        ⋮
 449 │        494  WHR          0.50996   0.512693  1.0152   0.984372
 450 │        495  WMB          0.502789  0.500684  1.01325  0.98707
 451 │        497  WYNN         0.480478  0.48463   1.0235   0.97825
 452 │        498  XEL          0.541833  0.530662  1.00859  0.99067
 453 │        500  XYL          0.525896  0.51166   1.01229  0.987488
 454 │        501  YUM          0.533865  0.518405  1.00933  0.990327
 455 │        502  ZBRA         0.534661  0.509754  1.01544  0.984313
 456 │        503  ZBH          0.502789  0.502663  1.01234  0.987887
 457 │        504  ZION         0.50757   0.50644   1.01554  0.984412
 458 │        505  ZTS          0.541833  0.518563  1.01068  0.988869
                                                      437 rows omitted

#### Save the computational results
Finally, we store the data in `binomial_model_parameter_table` as a `Comma Separated Value (CSV)` file in the `data` folder using the `CSV.write(...)` function, which is exported by the [CSV.jl](https://github.com/JuliaData/CSV.jl.git) package

In [15]:
CSV.write(joinpath(_PATH_TO_DATA,"binomial_parameters.csv"), binomial_model_parameter_table);